# Import

In [25]:
import pandas as pd

# Main ML library
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import sklearn as sk

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import psycopg2

import pandas as pd
from sqlalchemy import create_engine
import encodings



# Load Data

In [20]:
# Create connection to database

host = '157.230.240.246'
port = '5432'
username = 'iykra_trainee'
password = 'passwordiykra'
database = 'sandbox'

engine = create_engine ('postgresql+psycopg2://'+username+':'+password+'@'+host+':'+port+'/'+database)


In [22]:
df = pd.read_sql_query('select * from team5.z_user_tech_startup',con = engine)

In [23]:
df.head()

,user_id,age,gender,target
0,000329115b15395d42b7fce2fceb12fe74aa069800c0e9...,19,Male,Others
1,00046110472fcbcf8d0cc6dab24e1f48bda8015f35c733...,22,Female,Others
2,00086e681e2f1562a72071fa90cca05d44aa282ead5fbc...,15,Female,Others
3,000cbf1c85ea7bf75edd232355dec72ffbe6ceceb047fd...,20,Female,Others
4,000da74ac19c625e199c24545492d64a86054447a8256b...,29,Female,Others


## Split data 

In [26]:
train, test = train_test_split(df, test_size=0.2)

In [27]:
train.head()

,user_id,age,gender,target
23857,e955296127463a33ab2b4a6b34162e98436c4884d7405f...,15,Female,Others
17288,a8b80750a7965ec51441191c07e62f6c23151285b5428a...,15,Female,Others
4494,2bfd6ab88a392e0068e55f0e73237007c506a8ee836944...,27,Female,Others
3204,1ec56965d1a59e62304f3b993c22b0c23ce77337ded959...,15,Female,Others
106,0103d0bc9ab49dd43b016dca9c8cdeb79fa90238fcce1a...,21,Female,Others


In [2]:
trainFile = "C:/Users/adiwi/Documents/work/dataset/bank.csv"

In [3]:
train_pd = pd.read_csv(trainFile,sep=';')

In [4]:
train_pd.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


# Check Data 

In [5]:
# target data inbalance
train_pd.groupby(['y']).agg(['count'])

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
,count,count,count,count,count,count,count,count,count,count,count,count,count,count,count,count
y,,,,,,,,,,,,,,,,
no,4000,4000,4000,4000,4000,4000,4000,4000,4000,4000,4000,4000,4000,4000,4000,4000
yes,521,521,521,521,521,521,521,521,521,521,521,521,521,521,521,521


# Data Preprocessing

In [6]:

train_pd_processed = pd.get_dummies(train_pd, columns=['job','marital','education','default','housing','loan','contact','month','poutcome','y'])

In [7]:
train_pd_processed.head()

,age,balance,day,duration,campaign,pdays,previous,job_admin.,job_blue-collar,job_entrepreneur,...,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown,y_no,y_yes
0,30,1787,19,79,1,-1,0,0,0,0,...,0,0,1,0,0,0,0,1,1,0
1,33,4789,11,220,1,339,4,0,0,0,...,1,0,0,0,1,0,0,0,1,0
2,35,1350,16,185,1,330,1,0,0,0,...,0,0,0,0,1,0,0,0,1,0
3,30,1476,3,199,4,-1,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
4,59,0,5,226,1,-1,0,0,1,0,...,1,0,0,0,0,0,0,1,1,0


In [8]:
feature_names = list(train_pd_processed.columns)
do_not_use_for_training = ['job','marital','education','default','housing','loan','contact','month','poutcome','y','y_yes','y_no']

feature_names = [f for f in train_pd_processed.columns if f not in do_not_use_for_training]

print('Total features : {}'.format(len(feature_names)))

train_pd_processed[feature_names].count()
y = train_pd_processed['y_no']

Total features : 51


# Train Test

In [9]:
Xtr, Xv, ytr, yv = train_test_split(train_pd_processed[feature_names].values, y, test_size=0.3, random_state=1987)

# Modeling

In [10]:
RF = RandomForestClassifier()

In [11]:
RF.fit(Xtr, ytr)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

# Evaluation

In [12]:
trainPredict = RF.predict(Xtr)
obsPredict = RF.predict(Xv)

In [13]:
# Train Accuracy

RF.score(Xtr, ytr)

0.9962073324905183

In [14]:
# Test Accuracy

RF.score(Xv, yv)

0.8769344141488578

In [15]:
df_confusion = pd.crosstab(yv, obsPredict, rownames=['Actual'], colnames=['Predicted'], margins=True)

In [16]:
df_confusion

Predicted,0,1,All
Actual,,,
0,54,120,174
1,47,1136,1183
All,101,1256,1357


# Find data that's predicted '1' but actual '0'

In [17]:
train_pd.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [18]:
train_pd_processed = pd.get_dummies(train_pd, columns=['job','marital','education','default','housing','loan','contact','month','poutcome','y'])

In [19]:
feature_names = list(train_pd_processed.columns)
do_not_use_for_training = ['job','marital','education','default','housing','loan','contact','month','poutcome','y','y_yes','y_no']

feature_names = [f for f in train_pd_processed.columns if f not in do_not_use_for_training]

print('Total features : {}'.format(len(feature_names)))

train_pd_processed[feature_names].count()
y = train_pd_processed['y_no']

Total features : 51


In [20]:
allPreduct = RF.predict(train_pd_processed[feature_names])

In [21]:
train_pd['prediction'] = allPreduct

In [22]:
final_result = train_pd[(train_pd.y == 'no') & (train_pd.prediction==0)][['y','prediction']]

In [23]:
final_result.head()

,y,prediction
51,no,0
98,no,0
123,no,0
186,no,0
194,no,0


In [33]:
test = [["a",1],["b",2]]

In [35]:
a = []
for i in test:
    a.append(i[1])

In [36]:
a

[1, 2]